<center> <h1>PREDICIÓN DEL SUEÑO CON MLP</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>


###### Indice

En este jupyter notebook se lleva a cabo la construcción de un clasificador MLP de salida binaria para predecir el el estado de vigilia o sueño ligero y  el de sueño profundo.

* **1. Cargar datos Train-Test**
  * **1.1 Preparación datos de entrada y salida**
* **2. Clasificador MLP binario**
   * **2.1 Creación del modelo y selección parámetros libres**
   * **2.3 Evaluación del modelo**

IMPORTAR MÓDULOS

In [1]:
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import svm
import sklearn

## 1. CARGAR DATOS TRAIN-TEST

Se cargan los datos de train y test generados en los jupyter notebooks 'Nombre de los jupyter', en los que procesan los datos y se obtienen las características que se utilizan para entrenar los clasificadores. 

En el siguiente apartado se preparan los datos para seleccionar una salida binaria o multiclase según el clasificador que se vaya a utilizar.

### 2.1 Preparación datos de entrada y salida

In [2]:
Train= pd.read_csv('Train.csv')

In [3]:
Train

,Min_hr,Max_hr,Mean_hr,Men_acc_sqrt,Min_acc_x,Max_acc_x,Mean_acc_x,Min_acc_y,Max_acc_y,Mean_acc_y,Min_acc_z,Max_acc_z,Mean_acc_z,Etiquetas,Etiquetas binarias,ID
0,49.0,54.0,50.83,1.00,0.40,0.41,0.40,0.45,0.46,0.46,-0.80,-0.79,-0.79,0.0,0.0,0.0
1,59.0,59.0,59.00,0.99,-0.04,-0.03,-0.03,0.51,0.52,0.51,-0.86,-0.84,-0.85,0.0,0.0,0.0
2,52.0,53.0,52.83,0.99,-0.04,-0.03,-0.04,0.44,0.46,0.45,-0.90,-0.88,-0.88,1.0,1.0,0.0
3,56.0,59.0,58.00,0.99,-0.04,-0.04,-0.04,0.45,0.45,0.45,-0.89,-0.88,-0.88,2.0,1.0,0.0
4,61.0,61.0,61.00,0.99,-0.05,-0.04,-0.05,0.45,0.46,0.46,-0.88,-0.88,-0.88,3.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,52.0,59.0,55.67,1.00,-0.30,-0.29,-0.30,0.05,0.06,0.06,0.95,0.96,0.96,2.0,1.0,16.0
716,58.0,62.0,59.00,1.00,-0.30,-0.30,-0.30,0.05,0.06,0.06,0.95,0.96,0.96,2.0,1.0,16.0
717,53.0,65.0,57.40,1.00,-0.25,-0.24,-0.24,0.13,0.13,0.13,0.96,0.97,0.97,5.0,0.0,16.0
718,80.0,91.0,82.50,1.00,-0.54,-0.53,-0.54,-0.03,0.04,-0.01,0.84,0.85,0.85,1.0,1.0,16.0


In [4]:
#Train
Train= pd.read_csv('Train.csv')
#.drop(['Unnamed: 0'], axis = 1) a mi me los lee sin unnamed
Test= pd.read_csv('Test.csv')
#.drop(['Unnamed: 0'], axis = 1)

In [5]:
X_train=Train.drop(['Etiquetas', 'Etiquetas binarias','ID'], axis = 1) 
y_train_bin=Train['Etiquetas binarias']
y_train_multi=Train['Etiquetas']

X_test=Test.drop(['Etiquetas', 'Etiquetas binarias','ID'], axis = 1)
y_test_bin=Test['Etiquetas binarias']
y_test_multi=Test['Etiquetas']

## 2. CLASIFICADOR MLP BINARIO

Se ha utilizado un perceptrón multicapa para clasificar la vigilia o sueño ligero del sueño profundo. Es decir, se ha construido un clasificador con salida binaria.

### 2.1  Creación del modelo y selección parámetros libres

Se proponen posibles valores de neuronas en la capa oculta para ver con cuál de ellos se obtienen las mejores prestaciones para el modelo. Se crea una tabla que recoge la pérdidas para cada valor del número de neuronas y aquel que tenga menos pérdida es el que se utilizará para el conjunto de test.

In [6]:
# Número de neuronas de la capa oculta
n_hidden_1=2
n_hidden_2=3
n_hidden_3=4
n_hidden_4=5

n_hidden=[n_hidden_1, n_hidden_2, n_hidden_3, n_hidden_4]

In [7]:
# Tabla para recoger las prestaciones en train 
columns  = ['Loss','Score']
comp_train = pd.DataFrame(columns=columns)

for i in n_hidden:
    
    print('\033[4;32m' + 'MLP clasiffier with',i,'neurons in the hidden layer' + '\033[0m')

    model=MLPClassifier(activation='relu', batch_size=50, hidden_layer_sizes=(i,), learning_rate='constant',
        learning_rate_init=0.1, max_iter=30, shuffle=True, solver='sgd', tol=0.01, early_stopping=True,
        validation_fraction=0.2, verbose=True, n_iter_no_change=5)
    
    model.fit(X_train, y_train_bin)
    
    loss=model.loss_  
    comp_train = comp_train.set_value(i,('Loss','Score'),(loss,'Verbose'))

MLP clasiffier with 2 neurons in the hidden layer
Iteration 1, loss = 0.75593990
Validation score: 0.701389
Iteration 2, loss = 0.61495723
Validation score: 0.701389
Iteration 3, loss = 0.61350381
Validation score: 0.701389
Iteration 4, loss = 0.61306359
Validation score: 0.701389
Iteration 5, loss = 0.61220438
Validation score: 0.701389
Iteration 6, loss = 0.61160807
Validation score: 0.701389
Iteration 7, loss = 0.61187043
Validation score: 0.701389
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.
MLP clasiffier with 3 neurons in the hidden layer
Iteration 1, loss = 1.39026824
Validation score: 0.701389
Iteration 2, loss = 0.61850753
Validation score: 0.701389
Iteration 3, loss = 0.61537664
Validation score: 0.701389
Iteration 4, loss = 0.61554665
Validation score: 0.701389
Iteration 5, loss = 0.61474930
Validation score: 0.701389
Iteration 6, loss = 0.61530215
Validation score: 0.701389
Iteration 7, loss = 0.61575466
Validation score: 0.701

C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()
C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [8]:
comp_train

,Loss,Score
2,0.61187,Verbose
3,0.615755,Verbose
4,0.760835,Verbose
5,0.650933,Verbose


Las medidas de prestación elegidas para la elección de una topología han sido el validation score y la pérdida. Ambas se pueden ver al aplicar el fit del modelo, gracias al parámetro verbose=True. La pérdida la hemos añadido a la siguiente tabla pero la accuracy de cada epoch para el conjunto de validación hay que verla en la celda anterior. Según estos resultados, la mejor topología de MLP sería la de **n_hidden_1 con 5 neuronas** en la capa oculta, ya que el par (loss,val_score) es el mejor de los 4.
En todos los casos el valor 'validation score' está entorno a 0.66, pero el número de neuronas de menor pérdida es 5.

¿Cómo sabíamos si estaba memorizando o no?

### 2.2 Evaluación del modelo

Testeamos el modelo con el valor de número de neuronas en la capa ocula que ofrece la menor pérdida (número neuronas = 5)

In [9]:
columns  = ['Accuracy','Recall','F1-score']
comp_test = pd.DataFrame(columns=columns)

print('\033[4;32m' + 'MLP clasiffier with',n_hidden_1,'neurons in the hidden layer' + '\033[0m')

model=MLPClassifier(activation='relu', batch_size=50, hidden_layer_sizes=(n_hidden_1), learning_rate='constant',
    learning_rate_init=0.1, max_iter=30, shuffle=True, solver='sgd', tol=0.01, early_stopping=True,
    validation_fraction=0.2, verbose=True, n_iter_no_change=5)

model.fit(X_train,y_train_bin)

predictions = model.predict(X_test)
cr=classification_report(y_test_bin,predictions,output_dict=True)
acc,rec,f1=cr['weighted avg']['precision'],cr['weighted avg']['recall'],cr['weighted avg']['f1-score']

comp_test = comp_test.set_value(n_hidden_1, ('Accuracy','Recall','F1-score'),(acc,rec,f1))

MLP clasiffier with 2 neurons in the hidden layer
Iteration 1, loss = 0.73701933
Validation score: 0.701389
Iteration 2, loss = 0.61344198
Validation score: 0.701389
Iteration 3, loss = 0.61396544
Validation score: 0.701389
Iteration 4, loss = 0.61210856
Validation score: 0.701389
Iteration 5, loss = 0.61242954
Validation score: 0.701389
Iteration 6, loss = 0.61212017
Validation score: 0.701389
Iteration 7, loss = 0.61170668
Validation score: 0.701389
Validation score did not improve more than tol=0.010000 for 5 consecutive epochs. Stopping.


C:\Users\rober\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\rober\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [10]:
comp_test

,Accuracy,Recall,F1-score
2,0.551837,0.742857,0.633255


In [11]:
print("================Classification Report=================")
print(cr)

================Classification Report=================
{'0.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 54}, '1.0': {'precision': 0.7428571428571429, 'recall': 1.0, 'f1-score': 0.8524590163934427, 'support': 156}, 'accuracy': 0.7428571428571429, 'macro avg': {'precision': 0.37142857142857144, 'recall': 0.5, 'f1-score': 0.42622950819672134, 'support': 210}, 'weighted avg': {'precision': 0.5518367346938775, 'recall': 0.7428571428571429, 'f1-score': 0.6332552693208432, 'support': 210}}
